# Monte Carlo Tuning Analysis (v6)

This notebook is dedicated to evaluated the training for the v6 tuning using Monte Carlo 2015 samples setted to have the same detection as the cut based in the same sample. Let's check:

- Best tuning configuration;
- Tuning efficiencies using the best configuration;
- Training plots;
- Roc curve;
- And the best tuning configuration;

**NOTE**: The input files is storage in: `/Volumes/castor/tunings/RingerNote_2018/v6`

**NOTE**: All output files must be storage in: `/Volumes/castor/analysis/RingerNote_2018/tunings/v6`

## Import all necessary packages:

In [1]:
from saphyra.analysis import crossval_table
from Gaugi import load
import os, re, sys, glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import collections
%matplotlib inline
%config InlineBackend.figure_format='retina'

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence


## Create the tuning file dict:

In [2]:
def create_op_dict(op):
    d = {
              op+'_pd_ref'    : "reference/"+op+"_cutbased/pd_ref#0",
              op+'_fa_ref'    : "reference/"+op+"_cutbased/fa_ref#0",
              op+'_sp_ref'    : "reference/"+op+"_cutbased/sp_ref",
              op+'_pd_val'    : "reference/"+op+"_cutbased/pd_val#0",
              op+'_fa_val'    : "reference/"+op+"_cutbased/fa_val#0",
              op+'_sp_val'    : "reference/"+op+"_cutbased/sp_val",
              op+'_pd_op'     : "reference/"+op+"_cutbased/pd_op#0",
              op+'_fa_op'     : "reference/"+op+"_cutbased/fa_op#0",
              op+'_sp_op'     : "reference/"+op+"_cutbased/sp_op",
            
              # Counts
              op+'_pd_ref_passed'    : "reference/"+op+"_cutbased/pd_ref#1",
              op+'_fa_ref_passed'    : "reference/"+op+"_cutbased/fa_ref#1",
              op+'_pd_ref_total'     : "reference/"+op+"_cutbased/pd_ref#2",
              op+'_fa_ref_total'     : "reference/"+op+"_cutbased/fa_ref#2",   
              op+'_pd_val_passed'    : "reference/"+op+"_cutbased/pd_val#1",
              op+'_fa_val_passed'    : "reference/"+op+"_cutbased/fa_val#1",
              op+'_pd_val_total'     : "reference/"+op+"_cutbased/pd_val#2",
              op+'_fa_val_total'     : "reference/"+op+"_cutbased/fa_val#2",  
              op+'_pd_op_passed'     : "reference/"+op+"_cutbased/pd_op#1",
              op+'_fa_op_passed'     : "reference/"+op+"_cutbased/fa_op#1",
              op+'_pd_op_total'      : "reference/"+op+"_cutbased/pd_op#2",
              op+'_fa_op_total'      : "reference/"+op+"_cutbased/fa_op#2",
    } 
    return d

tuned_info = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
    
              #"loss"            : 'loss',
              #"val_loss"        : 'val_loss',
              #"accuracy"        : 'accuracy',
              #"val_accuracy"    : 'val_accuracy',
              #"max_sp_best_epoch_val": 'max_sp_best_epoch_val',
              } )

tuned_info.update(create_op_dict('tight'))
tuned_info.update(create_op_dict('medium'))
tuned_info.update(create_op_dict('loose'))
tuned_info.update(create_op_dict('vloose'))

## Open all tuning files:

In [5]:
cv = crossval_table( tuned_info )
cv.fill( '../tuning_data/v6/*/*/*.pic.gz', 'v6')
#!ls ../tuning_data/*

I0713 16:02:23.905680 140735835288448 macros.py:23] Reading file for v6 tag from ../tuning_data/v6/*/*/*.pic.gz
I0713 16:02:23.906816 140735835288448 macros.py:23] There are 800 files for this task...
I0713 16:02:23.909733 140735835288448 macros.py:23] Filling the table... 
I0713 16:02:38.867413 140735835288448 macros.py:23] End of fill step, a pandas DataFrame was created...


2020-07-13 16:02:23,905 | Py.crossval_table                       INFO Reading file for v6 tag from ../tuning_data/v6/*/*/*.pic.gz
2020-07-13 16:02:23,906 | Py.crossval_table                       INFO There are 800 files for this task...
2020-07-13 16:02:23,909 | Py.crossval_table                       INFO Filling the table... 
2020-07-13 16:02:38,867 | Py.crossval_table                       INFO End of fill step, a pandas DataFrame was created...


Let's keep only the best inits for each sort, configuration and eta/phi bin. To calculate this we must choose an evaluation method to keep the max value in each configuration. Here, we will get the best inists looking for the max SP value for each one.

In [6]:
best_inits = cv.filter_inits("max_sp_val")
best_inits.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_passed,vloose_fa_val_passed,vloose_pd_val_total,vloose_fa_val_total,vloose_pd_op_passed,vloose_fa_op_passed,vloose_pd_op_total,vloose_fa_op_total
14,v6,0,0,0,0,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.976299,0.979675,...,4916,332468,487,1619,492,33247,4869,16715,4916,332468
117,v6,0,0,0,1,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.977713,0.975610,...,4916,332468,487,1884,492,33247,4869,15813,4916,332468
71,v6,0,0,0,2,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.977519,0.979675,...,4916,332468,487,1477,492,33247,4869,16134,4916,332468
145,v6,0,0,0,3,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.973329,0.979675,...,4916,332468,487,1724,492,33247,4869,15812,4916,332468
85,v6,0,0,0,4,0,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.978614,0.989837,...,4916,332468,487,1082,492,33246,4869,15466,4916,332468


## Fix the reference counts:

In [7]:
def fix_counts(table, op):
    table['%s_fa_ref_total'%op]  = table['%s_fa_op_total'%op]
    table['%s_pd_ref_total'%op]  = table['%s_pd_op_total'%op]
    table['%s_fa_ref_passed'%op] = table['%s_fa_ref'%op] * table['%s_fa_op_total'%op]
    table['%s_pd_ref_passed'%op] = table['%s_pd_ref'%op] * table['%s_pd_op_total'%op]
    
for op in ['tight','medium','loose','vloose']:
    fix_counts(best_inits, op)


In [8]:
best_inits.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_passed,vloose_fa_val_passed,vloose_pd_val_total,vloose_fa_val_total,vloose_pd_op_passed,vloose_fa_op_passed,vloose_pd_op_total,vloose_fa_op_total
14,v6,0,0,0,0,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.976299,0.979675,...,4916,332468,487,1619,492,33247,4869,16715,4916,332468
117,v6,0,0,0,1,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.977713,0.975610,...,4916,332468,487,1884,492,33247,4869,15813,4916,332468
71,v6,0,0,0,2,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.977519,0.979675,...,4916,332468,487,1477,492,33247,4869,16134,4916,332468
145,v6,0,0,0,3,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.973329,0.979675,...,4916,332468,487,1724,492,33247,4869,15812,4916,332468
85,v6,0,0,0,4,0,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.978614,0.989837,...,4916,332468,487,1082,492,33246,4869,15466,4916,332468


## Plot the evolution for each configuration

Here, each configurations means the number of neurons in the hidden layer. The tuning proceeding follow this topology:

- First layer with 100 inputs;
- Second layer with 2 to 10 neurons (all with hyperbolic tangent);
- One output with hyperbolic tangent.

Let's check the SP, Fake and detection evolution for each layer configuration calculating the mean and std values for all 10 best inits (1 init per sort, 10 sorts, 10 values per configuration).

In [11]:

def plot_evolution( t , et_bin, eta_bin , output, display=False):

    sp_val_mean = []; sp_val_std = []
    for model_idx in t.model_idx.unique():
        table = t.loc[ (t.model_idx==model_idx) & (t.et_bin==et_bin) & (t.eta_bin==eta_bin)]    
        sp_val_mean.append( table['max_sp_val'].mean() * 100)
        sp_val_std.append( table['max_sp_val'].std() * 100)
    neurons = [ i+2 for i in range(len(sp_val_mean))]
    fig, ax = plt.subplots(1,1, figsize=(15,5))
    plt.errorbar(neurons, sp_val_mean, yerr=sp_val_std, label='Max SP (validation)')
    ax.set(xlabel='#neurons', ylabel='SP',
       title='Max SP in the validation set (et=%d, eta=%d)'%(et_bin,eta_bin))
    ax.grid()
    print('saving %s...'%output)
    plt.savefig(output)
    if display:
        plt.show()      
    else:
        plt.close(fig)

        
for et_bin in best_inits.et_bin.unique():
    for eta_bin in best_inits.eta_bin.unique():
        plot_evolution(best_inits, et_bin, eta_bin, 'sp_evolution_per_config_et%d_eta%d.pdf'%(et_bin,eta_bin))

saving sp_evolution_per_config_et0_eta0.pdf...
saving sp_evolution_per_config_et0_eta1.pdf...
saving sp_evolution_per_config_et0_eta2.pdf...
saving sp_evolution_per_config_et0_eta3.pdf...
saving sp_evolution_per_config_et1_eta0.pdf...
saving sp_evolution_per_config_et1_eta1.pdf...
saving sp_evolution_per_config_et1_eta2.pdf...
saving sp_evolution_per_config_et1_eta3.pdf...
saving sp_evolution_per_config_et2_eta0.pdf...
saving sp_evolution_per_config_et2_eta1.pdf...
saving sp_evolution_per_config_et2_eta2.pdf...
saving sp_evolution_per_config_et2_eta3.pdf...
saving sp_evolution_per_config_et3_eta0.pdf...
saving sp_evolution_per_config_et3_eta1.pdf...
saving sp_evolution_per_config_et3_eta2.pdf...
saving sp_evolution_per_config_et3_eta3.pdf...
saving sp_evolution_per_config_et4_eta0.pdf...
saving sp_evolution_per_config_et4_eta1.pdf...
saving sp_evolution_per_config_et4_eta2.pdf...
saving sp_evolution_per_config_et4_eta3.pdf...


For simplicity, let's keep the five neurons (model id = 3) in the hidden layer for all phase spaces.

## Calculate the cross validation table for 5 neurons for all phase spaces

In [12]:
best_inits = best_inits.loc[best_inits.model_idx==3]
best_inits.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_passed,vloose_fa_val_passed,vloose_pd_val_total,vloose_fa_val_total,vloose_pd_op_passed,vloose_fa_op_passed,vloose_pd_op_total,vloose_fa_op_total
17,v6,0,0,3,0,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.976215,0.983740,...,4916,332468,487,1824,492,33247,4869,18569,4916,332468
79,v6,0,0,3,1,0,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.977517,0.975610,...,4916,332468,487,1880,492,33247,4869,15925,4916,332468
74,v6,0,0,3,2,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.977414,0.979675,...,4916,332468,487,1503,492,33247,4869,16193,4916,332468
157,v6,0,0,3,3,0,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.972286,0.977642,...,4916,332468,487,1782,492,33247,4869,15841,4916,332468
88,v6,0,0,3,4,0,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.978571,0.987805,...,4916,332468,487,1442,492,33246,4869,15713,4916,332468


In [13]:
# 20 bins X 10 sorts = 200 rows
best_inits.shape

(200, 98)

In [14]:
cv.describe(best_inits)

,train_tag,et_bin,eta_bin,max_sp_val_mean,max_sp_val_std,max_sp_pd_val_mean,max_sp_pd_val_std,max_sp_fa_val_mean,max_sp_fa_val_std,max_sp_op_mean,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_total_mean,vloose_pd_val_total_std,vloose_fa_val_total_mean,vloose_fa_val_total_std,vloose_pd_op_total_mean,vloose_pd_op_total_std,vloose_fa_op_total_mean,vloose_fa_op_total_std
0,v6,0,0,0.976775,0.001971,0.983525,0.005017,0.029943,0.004397,0.974425,...,4916,332468,491.6,0.516398,33246.8,0.421637,4916.0,0.0,332468.0,0.0
1,v6,0,1,0.968289,0.003721,0.979386,0.007646,0.042731,0.005363,0.964453,...,3008,253818,300.8,0.421637,25381.8,0.421637,3008.0,0.0,253818.0,0.0
2,v6,0,2,0.948774,0.010512,0.969730,0.015329,0.071865,0.018676,0.938986,...,1155,48521,115.5,0.527046,4852.1,0.316228,1155.0,0.0,48521.0,0.0
3,v6,0,3,0.961576,0.002894,0.976531,0.009260,0.053237,0.006779,0.957620,...,5027,372296,502.7,0.483046,37229.6,0.516398,5027.0,0.0,372296.0,0.0
4,v6,1,0,0.984509,0.000925,0.990404,0.002402,0.021366,0.002491,0.984029,...,32722,168165,3272.2,0.421637,16816.5,0.527046,32722.0,0.0,168165.0,0.0
5,v6,1,1,0.978097,0.001719,0.984946,0.004246,0.028724,0.002257,0.977518,...,19264,129473,1926.4,0.516398,12947.3,0.483046,19264.0,0.0,129473.0,0.0
6,v6,1,2,0.958565,0.004029,0.979444,0.010082,0.062047,0.010438,0.955915,...,6033,21806,603.3,0.483046,2180.6,0.516398,6033.0,0.0,21806.0,0.0
7,v6,1,3,0.970092,0.001525,0.984203,0.004522,0.043913,0.002667,0.968975,...,24688,177079,2468.8,0.421637,17707.9,0.316228,24688.0,0.0,177079.0,0.0
8,v6,2,0,0.990516,0.001223,0.993777,0.001375,0.012738,0.003261,0.990273,...,87257,34622,8725.7,0.483046,3462.2,0.421637,87257.0,0.0,34622.0,0.0
9,v6,2,1,0.985577,0.000934,0.991707,0.001886,0.020532,0.002148,0.985155,...,53175,27420,5317.5,0.527046,2742.0,0.000000,53175.0,0.0,27420.0,0.0


## Create the beamer table

In [15]:
# Create beamer presentation
for op in ['tight','medium','loose','vloose']:
    cv.dump_beamer_table( best_inits ,  [15,20,25,30,40,50], 
                     [0, 0.8 , 1.37, 1.54, 2.5], 
                     [op],
                     'tuning_v6_'+op,
                     title = op+' Tunings (v6)',
                     tags = ['v6'],
                    )


I0713 16:12:17.693154 140735835288448 BeamerAPI.py:475] Started creating beamer file tuning_v6_tight.pdf latex code...
I0713 16:12:22.671333 140735835288448 BeamerAPI.py:475] Started creating beamer file tuning_v6_medium.pdf latex code...
I0713 16:12:26.796303 140735835288448 BeamerAPI.py:475] Started creating beamer file tuning_v6_loose.pdf latex code...
I0713 16:12:31.961796 140735835288448 BeamerAPI.py:475] Started creating beamer file tuning_v6_vloose.pdf latex code...


2020-07-13 16:12:17,693 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v6_tight.pdf latex code...
2020-07-13 16:12:22,671 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v6_medium.pdf latex code...
2020-07-13 16:12:26,796 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v6_loose.pdf latex code...
2020-07-13 16:12:31,961 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v6_vloose.pdf latex code...


## Plot all Training for each sort

In [17]:

def plot_training_curves_for_each_sort(table, et_bin, eta_bin, output, display=False):
    
    table = table.loc[(table.et_bin==et_bin) & (table.eta_bin==eta_bin)] 
    nsorts = len(table.sort.unique())
    fig, ax = plt.subplots(nsorts,2, figsize=(15,20))
    fig.suptitle(r'Monitoring Train Plot - Et = %d, Eta = %d'%(et_bin,eta_bin), fontsize=15)
    for idx, sort in enumerate(table.sort.unique()):
        current_table = table.loc[table.sort==sort]
        path=current_table.file_name.values[0]
        history = load(path)['tunedData'][current_table.model_idx.values[0]]['history']
        best_epoch = history['max_sp_best_epoch_val'][-1]
        # Make the plot here
        ax[idx, 0].set_xlabel('Epochs')
        ax[idx, 0].set_ylabel('Loss')
        ax[idx, 0].plot(history['loss'], c='b', label='Train Step')
        ax[idx, 0].plot(history['val_loss'], c='r', label='Validation Step') 
        ax[idx, 0].axvline(x=best_epoch, c='k', label='Best epoch')
        ax[idx, 0].legend()
        ax[idx, 0].grid()
        ax[idx, 1].set_xlabel('Epochs')
        ax[idx, 1].set_ylabel('SP')
        ax[idx, 1].plot(history['max_sp_val'], c='r', label='Validation Step') 
        ax[idx, 1].axvline(x=best_epoch, c='k', label='Best epoch')
        ax[idx, 1].legend()
        ax[idx, 1].grid()
    
    print(output)
    plt.savefig(output)
    if display:
        plt.show()
    else:
        plt.close(fig)
    
for et_bin in best_inits.et_bin.unique():
    for eta_bin in best_inits.eta_bin.unique():
        plot_training_curves_for_each_sort(best_inits,et_bin, eta_bin, 'train_evolution_best_config_et%d_eta%d.pdf'%(et_bin,eta_bin))

train_evolution_best_config_et0_eta0.pdf
train_evolution_best_config_et0_eta1.pdf
train_evolution_best_config_et0_eta2.pdf
train_evolution_best_config_et0_eta3.pdf
train_evolution_best_config_et1_eta0.pdf
train_evolution_best_config_et1_eta1.pdf
train_evolution_best_config_et1_eta2.pdf
train_evolution_best_config_et1_eta3.pdf
train_evolution_best_config_et2_eta0.pdf
train_evolution_best_config_et2_eta1.pdf
train_evolution_best_config_et2_eta2.pdf
train_evolution_best_config_et2_eta3.pdf
train_evolution_best_config_et3_eta0.pdf
train_evolution_best_config_et3_eta1.pdf
train_evolution_best_config_et3_eta2.pdf
train_evolution_best_config_et3_eta3.pdf
train_evolution_best_config_et4_eta0.pdf
train_evolution_best_config_et4_eta1.pdf
train_evolution_best_config_et4_eta2.pdf
train_evolution_best_config_et4_eta3.pdf


## Check the integrate value:

Here we will check the integrated value:

In [18]:
cv.integrate(best_inits, 'v6') * 100

,tight_pd_ref,tight_fa_ref,tight_pd_val,tight_fa_val,tight_pd_op,tight_fa_op,medium_pd_ref,medium_fa_ref,medium_pd_val,medium_fa_val,...,loose_pd_val,loose_fa_val,loose_pd_op,loose_fa_op,vloose_pd_ref,vloose_fa_ref,vloose_pd_val,vloose_fa_val,vloose_pd_op,vloose_fa_op
mean,9.929511e+01,16.683031,99.315703,4.552510,99.295956,4.747503,99.391064,1.812587e+01,99.404795,4.712839,...,99.680248,5.875439,99.677975,6.148168,99.707547,30.269198,99.711961,6.221100,99.707694,6.376476
std,1.170278e-14,0.000000,0.017179,0.372819,0.001300,0.064910,0.000000,2.925695e-15,0.015058,0.412961,...,0.008078,0.634247,0.000439,0.070564,0.000000,0.000000,0.006883,0.778065,0.001118,0.078311


## Get the Best sort for all phase spaces:


Here we will access the best sort for each phase space. We can use this information to export each tuning in each phase space to a unique file that will be used in the pileup adjustiment step (using prometheus framework). We will get the best sort with the highest max SP value for each phase space always.

In [21]:
best_sorts = cv.filter_sorts(best_inits, 'max_sp_val')
print(best_sorts.shape)
best_sorts.head(20)

(20, 98)


,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_passed,vloose_fa_val_passed,vloose_pd_val_total,vloose_fa_val_total,vloose_pd_op_passed,vloose_fa_op_passed,vloose_pd_op_total,vloose_fa_op_total
97,v6,0,0,3,5,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.979432,0.987805,...,4916,332468,487,1226,492,33246,4869,16336,4916,332468
291,v6,0,1,3,4,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.974170,0.990033,...,3008,253818,299,1229,301,25382,2986,22143,3008,253818
503,v6,0,2,3,8,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.965494,0.991304,...,1155,48521,114,291,115,4852,1141,6641,1155,48521
651,v6,0,3,3,4,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.966410,0.988072,...,5027,372296,495,1995,503,37229,4948,27716,5027,372296
799,v6,1,0,3,1,0,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.985777,0.990834,...,32722,168165,3259,591,3273,16816,32583,5505,32722,168165
1062,v6,1,1,3,0,0,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.981986,0.992216,...,19264,129473,1921,528,1927,12947,19203,7793,19264,129473
1159,v6,1,2,3,1,0,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.963639,0.983444,...,6033,21806,586,112,604,2180,5867,1304,6033,21806
1390,v6,1,3,3,9,0,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.973152,0.990276,...,24688,177079,2451,951,2468,17708,24516,10686,24688,177079
1457,v6,2,0,3,0,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.992662,0.993124,...,87257,34622,8713,75,8726,3462,87111,850,87257,34622
1632,v6,2,1,3,7,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,3,0.987409,0.994170,...,53175,27420,5314,157,5317,2742,53142,1694,53175,27420


## Dump all best tunings:

Here we will dump all 20 configuration into a json file. We use the same neural netowrks for each operation point. Only the thresholds are variated. The threshold correction (alpha and beta) and files will be calculated/generated using the pileup correction tool.

In [37]:

def dump_models( best_sorts , etbins, etabins, mubins , output):

    from saphyra.layers.RpLayer import RpLayer
    # Just to remove the keras dependence
    import tensorflow as tf
    model_from_json = tf.keras.models.model_from_json
    import json
    
    d = {
        "models"          : [],
        "__version__"     : "v6", 
        "__type__"        : "Model", 
        "__name__"        : "TrigL2_20170505_v6", 
        "__description__" : "", 
        "__operation__"   : "",
     }

    def _treat_weights( weights ):
        for i, w in enumerate(weights):
            weights[i]=w.tolist()
        return weights

    for et_bin in range(len(etbins)):
        for et_bin in range(len(etabins)):
            d_tuned = {}
            best = best_sorts.loc[(best_sorts.et_bin==et_bin) & (best_sorts.eta_bin==eta_bin)]
            best.head()
            tuned = load(best.file_name.values[0])['tunedData'][best.model_idx.values[0]]
            #model = model_from_json( json.dumps(tuned['sequence'], separators=(',', ':')) , 
            #                     custom_objects={'RpLayer':RpLayer} )
            #model.set_weights( tuned['weights'] )
            #model.summary()
            d_tuned['sequence'] = tuned['sequence']
            d_tuned['weights']  = _treat_weights(tuned['weights'])
            d_tuned['etBin']    = [etbins[et_bin], etbins[et_bin+1]]
            d_tuned['etaBin']   = [etabins[eta_bin], etabins[eta_bin+1]]
            d_tuned['muBin']    = mubins
            d['models'].append(d_tuned)
            
    with open(output,"w") as f:
        print('Dump tunings: %s'%output)
        json.dump(d,f)
            
        
        


etbins = [15,20,25,30,40,50,100000]
etabins = [0, 0.8 , 1.37, 1.54, 2.5]
mubins = [0,100]


dump_models( best_sorts, etbins, etabins, mubins, 'test.json')



Dump tunings: test.json
